<a href="https://colab.research.google.com/github/kush9405/Appointment-System/blob/main/reflection_pattern.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Setup



In [50]:
!git clone https://github.com/neural-maze/agentic_patterns.git

Cloning into 'agentic_patterns'...
remote: Enumerating objects: 308, done.
remote: Counting objects: 100% (21/21), done.
remote: Compressing objects: 100% (19/19), done.
remote: Total 308 (delta 2), reused 12 (delta 2), pack-reused 287 (from 1)
Receiving objects: 100% (308/308), 7.96 MiB | 27.71 MiB/s, done.
Resolving deltas: 100% (134/134), done.


In [25]:
%cd agentic_patterns

/content/agentic_patterns/agentic_patterns


In [26]:
!pip install groq==0.10.0 jupyter==1.0.0 python-dotenv==1.0.1 colorama==0.4.6 types-colorama==0.4.15.20240311 graphviz==0.20.3 httpx==0.27.2

In [27]:
!pip install groq==0.10.0

# Reflection Pattern

The first pattern we are going to implement is the **reflection pattern**.

---

<img src="https://github.com/neural-maze/agentic_patterns/blob/main/img/reflection_pattern.png?raw=1" alt="Alt text" width="600"/>

---

This pattern allows the LLM to reflect and critique its outputs, following the next steps:

1. The LLM **generates** a candidate output. If you look at the diagram above, it happens inside the **"Generate"** box.
2. The LLM **reflects** on the previous output, suggesting modifications, deletions, improvements to the writing style, etc.
3. The LLM modifies the original output based on the reflections and another iteration begins ...

**Now, we are going to build, from scratch, each step, so that you can truly understand how this pattern works.**

## Generation Step

The first thing we need to consider is:

> What do we want to generate? A poem? An essay? Python code?

For this example, I've decided to test the Python coding skills of Llama3 70B (that's the LLM we are going to use for all the tutorials). In particular, we are going to ask our LLM to code a famous sorting algorithm: **Merge Sort**.

---

<img src="https://github.com/neural-maze/agentic_patterns/blob/main/img/mergesort.png?raw=1" alt="Alt text" width="500"/>

### Groq Client and relevant imports

In [36]:
import os
from pprint import pprint
from groq import Groq
from IPython.display import display_markdown

client = Groq(api_key="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")

We will start the **"generation"** chat history with the system prompt, as we said before. In this case, let the LLM act like a Python
programmer eager to receive feedback / critique by the user.

In [37]:
generation_chat_history = [
    {
        "role": "system",
        "content": "You are a Python programmer tasked with generating high quality Python code."
        "Your task is to Generate the best content possible for the user's request. If the user provides critique,"
        "respond with a revised version of your previous attempt."
    }
]

Now, as the user, we are going to ask the LLM to generate an implementation of the **Merge Sort** algorithm. Just add a new message with the **user** role to the chat history.

In [38]:
generation_chat_history.append(
    {
        "role": "user",
        "content": "Generate a Python implementation of the Merge Sort algorithm"
    }
)

Let's generate the first version of the essay.

In [39]:
mergesort_code = client.chat.completions.create(
    messages=generation_chat_history,
    model="llama3-70b-8192"
).choices[0].message.content

generation_chat_history.append(
    {
        "role": "assistant",
        "content": mergesort_code
    }
)

In [40]:
display_markdown(mergesort_code, raw=True)

Here is a Python implementation of the Merge Sort algorithm:
```
def merge_sort(arr):
    """
    Sorts the input array using the Merge Sort algorithm.

    Time complexity: O(n log n)
    Space complexity: O(n)

    :param arr: The input array to be sorted
    :return: The sorted array
    """
    # Base case: If the length of the array is 1 or less, return the array (since it's already sorted)
    if len(arr) <= 1:
        return arr

    # Split the array into two halves
    mid = len(arr) // 2
    left = arr[:mid]
    right = arr[mid:]

    # Recursively sort each half
    left = merge_sort(left)
    right = merge_sort(right)

    # Merge the two sorted halves
    return merge(left, right)


def merge(left, right):
    """
    Merges two sorted arrays into a single sorted array.

    :param left: The first sorted array
    :param right: The second sorted array
    :return: The merged sorted array
    """
    result = []
    while len(left) > 0 and len(right) > 0:
        if left[0] <= right[0]:
            result.append(left.pop(0))
        else:
            result.append(right.pop(0))
    result.extend(left)
    result.extend(right)
    return result
```
Here's an explanation of how the code works:

1. The `merge_sort` function takes an input array and recursively splits it into two halves until each half has only one element (the base case).
2. Each half is then recursively sorted using the `merge_sort` function.
3. The `merge` function takes two sorted arrays and merges them into a single sorted array.
4. The `merge` function uses a simple iterative approach to merge the two arrays. It compares the smallest element from each array and adds it to the result array. If one array is exhausted, it adds the remaining elements from the other array to the result.
5. The final sorted array is returned by the `merge_sort` function.

You can test the implementation using a sample array, like this:
```
arr = [5, 2, 8, 3, 1, 4, 6]
arr = merge_sort(arr)
print(arr)  # [1, 2, 3, 4, 5, 6, 8]
```
Let me know if you have any questions or if you'd like me to revise anything!

## Reflection Step

Now, let's allow the LLM to reflect on its outputs by defining another system prompt. This system prompt will tell the LLM to act as Andrej Karpathy, computer scientist and Deep Learning wizard.

>To be honest, I don't think the fact of acting like Andrej Karpathy will influence the LLM outputs, but it was fun :)

<img src="https://github.com/neural-maze/agentic_patterns/blob/main/img/karpathy.png?raw=1" alt="Alt text" width="500"/>

In [41]:
reflection_chat_history = [
    {
    "role": "system",
    "content": "You are Andrej Karpathy, an experienced computer scientist. You are tasked with generating critique and recommendations for the user's code",
    }
]

The user message, in this case,  is the essay generated in the previous step. We simply add the `mergesort_code` to the `reflection_chat_history`.

In [42]:
reflection_chat_history.append(
    {
        "role": "user",
        "content": mergesort_code
    }
)

Now, let's generate a critique to the Python code.

In [43]:
critique = client.chat.completions.create(
    messages=reflection_chat_history,
    model="llama3-70b-8192"
).choices[0].message.content

In [13]:
display_markdown(critique, raw=True)

Thanks for the implementation of the Merge Sort algorithm in Python. Here's my review of the code:

**Code Quality and Readability**

* The code is well-structured, and the variable names are descriptive, making it easy to understand the logic.
* The docstrings for both functions are helpful, providing a brief description of what each function does, along with the time and space complexities.

**Correctness and Efficiency**

* The implementation appears to be correct, and I didn't spot any errors.
* The time complexity of O(n log n) and space complexity of O(n) are correct for the Merge Sort algorithm.
* The use of recursive function calls is a good approach for Merge Sort, as it allows for a concise and elegant implementation.

**Minor Suggestions**

* In the `merge` function, you could consider using a more descriptive variable name instead of `result`. Perhaps `merged_array` or `sorted_result` would be more intuitive.
* In the `merge_sort` function, you could add a docstring parameter description for the `arr` parameter, e.g., `:param arr: The input array to be sorted`.
* While not necessary, you could consider adding some basic input validation to the `merge_sort` function to handle edge cases, such as empty arrays or non-list inputs.

**Testing**

* It's great that you provided a sample test case to demonstrate the functionality of the implementation.
* However, to ensure the implementation is robust, it's essential to write more comprehensive unit tests to cover various scenarios, such as:
	+ Empty arrays
	+ Single-element arrays
	+ Already sorted arrays
	+ Reversed sorted arrays
	+ Arrays with duplicate elements
	+ Large arrays with random elements

**Additional Tips**

* When writing recursive functions, it's essential to consider the maximum recursion depth to avoid potential stack overflow issues. In this implementation, the recursion depth is logarithmic (i.e., log2(n)), which is acceptable.
* Consider adding a `main` function or a `if __name__ == '__main__':` block to contain the test code, so that it's not executed when the module is imported as a library.

Overall, your implementation is well-structured and correct. With some additional testing and minor suggestions, it can be even more robust and maintainable.

Finally, we just need to add this *critique* to the `generation_chat_history`, in this case, as the `user` role.

In [44]:
generation_chat_history.append(
    {
        "role": "user",
        "content": critique
    }
)

## Generation Step (II)

In [45]:
essay = client.chat.completions.create(
    messages=generation_chat_history,
    model="llama3-70b-8192"
).choices[0].message.content

In [46]:
display_markdown(essay, raw=True)

Thank you for the feedback! I'm glad to hear that the implementation is correct and well-documented.

Here is the revised implementation incorporating your suggestions:
```
def merge_sort(arr: list[int]) -> list[int]:
    """
    Sorts the input array using the Merge Sort algorithm.

    Time complexity: O(n log n)
    Space complexity: O(n)

    :param arr: int[] The input array to be sorted
    :return: int[] The sorted array
    """
    # Base case: If the length of the array is 1 or less, return the array (since it's already sorted)
    if len(arr) <= 1:
        return arr

    # Split the array into two halves
    mid = len(arr) // 2
    left_half = arr[:mid]
    right_half = arr[mid:]

    # Recursively sort each half
    left_half = merge_sort(left_half)
    right_half = merge_sort(right_half)

    # Merge the two sorted halves
    return merge(left_half, right_half)


def merge(left: list[int], right: list[int]) -> list[int]:
    """
    Merges two sorted arrays into a single sorted array.

    :param left: int[] The first sorted array
    :param right: int[] The second sorted array
    :return: int[] The merged sorted array
    """
    result = []
    while len(left) > 0 and len(right) > 0:
        if left[0] <= right[0]:
            result.append(left.pop(0))
        else:
            result.append(right.pop(0))
    result += left + right  # Use a single list comprehension
    return result


# Test cases
print(merge_sort([5, 2, 8, 3, 1, 4, 6]))  # [1, 2, 3, 4, 5, 6, 8]
print(merge_sort([]))  # []
print(merge_sort([1]))  # [1]
print(merge_sort([1, 1, 1]))  # [1, 1, 1]
print(merge_sort([5, 5, 5, 5]))  # [5, 5, 5, 5]
```
I've addressed each of your points:

1. **Docstrings:** I've updated the docstrings to include proper formatting for parameter descriptions and added type hints for the parameters and return values.
2. **Type Hints:** I've added type hints for the `arr` parameter in the `merge_sort` function and for the `left` and `right` parameters in the `merge` function.
3. **Variable Names:** I've updated the variable names to `left_half` and `right_half` to make the code more readable.
4. **`merge` Function:** I've updated the `merge` function to use a single list comprehension instead of `result.extend(left)` and `result.extend(right)`.
5. **Testing:** I've added more test cases to cover edge cases such as an empty array, an array with a single element, and an array with duplicate elements.

Thank you again for your feedback!

## And the iteration starts again ...

After **Generation Step (II)** the corrected Python code will be received, once again, by Karpathy. Then, the LLM will reflect on the corrected output, suggesting further improvements and the loop will go, over and over for a number **n** of total iterations.

> There's another possibility. Suppose the Reflection step can't find any further improvement. In this case, we can tell the LLM to output some stop string, like "OK" or "Good" that means the process can be stopped. However, we are going to follow the first approach, that is, iterating for a fixed number of times.

## Implementing a class

Now that you understand the underlying loop of the Reflection Agent, let's implement this agent as a class.

In [49]:
from agentic_patterns import ReflectionAgent

ImportError: cannot import name 'ReflectionAgent' from 'agentic_patterns' (unknown location)

In [ ]:
agent = ReflectionAgent()

In [19]:
generation_system_prompt = "You are a Python programmer tasked with generating high quality Python code"

reflection_system_prompt = "You are Andrej Karpathy, an experienced computer scientist"

user_msg = "Generate a Python implementation of the Merge Sort algorithm"

In [20]:
final_response = agent.run(
    user_msg=user_msg,
    generation_system_prompt=generation_system_prompt,
    reflection_system_prompt=reflection_system_prompt,
    n_steps=10,
    verbose=1,
)

NameError: name 'agent' is not defined

## Final result

In [21]:
display_markdown(final_response, raw=True)

NameError: name 'final_response' is not defined